In [5]:
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
import os

def cropVideo(src, dest, x,y,h,w):
    clip = VideoFileClip(src)
    cropped_clip = crop(clip, x1=x, y1=y, width=w, height=h)
    cropped_clip.write_videofile(dest)

# cropVideo(r"C:\Users\Dell\Downloads\Did you know Obsidian isn't just a rock It's a glimpse into our volcanic past! 🔥 #DidYouKnow #ObsidianFacts #Geology101.mp4", r"media\Did you know Obsidian isn't just a rock It's a glimpse into our volcanic past! 🔥 #DidYouKnow #ObsidianFacts #Geology101.mp4", 69,244,1676,942)
title = "Do you know! Cholera can be deadly, but we can fight it with clean water and sanitation. 💧 #DidYouKnow #Cholera #PublicHealth"
title.replace(": ", " ")
downloadName = "Do you know- Cholera-1713704344"
recent_mp4 = os.path.join(os.path.expanduser("~/Downloads"), downloadName + ".mp4")

current_directory = os.path.join(os.getcwd(), "media")
path = os.path.join(current_directory, title + ".mp4")
cropVideo(recent_mp4, path, 69, 244, 1676, 942)
os.remove(recent_mp4)

Moviepy - Building video d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Do you know! Cholera can be deadly, but we can fight it with clean water and sanitation. 💧 #DidYouKnow #Cholera #PublicHealth.mp4.
MoviePy - Writing audio in Do you know! Cholera can be deadly, but we can fight it with clean water and sanitation. 💧 #DidYouKnow #Cholera #PublicHealthTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Do you know! Cholera can be deadly, but we can fight it with clean water and sanitation. 💧 #DidYouKnow #Cholera #PublicHealth.mp4



Moviepy - Done !
Moviepy - video ready d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Do you know! Cholera can be deadly, but we can fight it with clean water and sanitation. 💧 #DidYouKnow #Cholera #PublicHealth.mp4


In [7]:
from moviepy.editor import VideoFileClip
from moviepy.video.fx.all import crop
import os
import threading

def cropVideo(src, dest, x, y, h, w):
    clip = VideoFileClip(src)
    cropped_clip = crop(clip, x1=x, y1=y, width=w, height=h)
    cropped_clip.write_videofile(dest)

def process_video(title_info):
    title = title_info['title']
    downloadName = title_info['download_name']
    recent_mp4 = os.path.join(os.path.expanduser("~/Downloads"), downloadName + ".mp4")
    
    current_directory = os.path.join(os.getcwd(), "media")
    dest_path = os.path.join(current_directory, title + ".mp4")
    
    cropVideo(recent_mp4, dest_path, title_info['x'], title_info['y'], title_info['h'], title_info['w'])
    os.remove(recent_mp4)

def multi_thread_crop(titles_info):
    threads = []
    for title_info in titles_info:
        thread = threading.Thread(target=process_video, args=(title_info,))
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()

# Read titles from a file (assuming each line contains title info in the format: title,downloadName,x,y,h,w)
titles_info = []
with open("titles.txt", "r", encoding="utf-8") as f:
    for line in f.read().split("\n"):
        title_info = line.split(":")
        print(title_info)
        titles_info.append({
            'title': title_info[0],
            'download_name': title_info[1],
            'x': 69,
            'y': 244,
            'h': 1676,
            'w': 942
        })

multi_thread_crop(titles_info)

["Spices aren't just for cooking—they've shaped history! 🌶️🧭 #DidYouKnow #SpiceTrade #History #FoodCulture", 'KLr7XS6MxnMJ-1713728158']
['Discover the hidden facts about your favorite fictional characters! #DidYouKnow #FictionalCharacters #FunFacts', 'Discover the hidden -1713728080']
['From ARPANET to the World Wide Web, the internet has a fascinating history! #DidYouKnow #InternetHistory #Technology', 'From ARPANET to the -1713728045']
['Discover the fascinating history behind Mercedes-Benz, from its name to its innovative safety features! #MercedesBenz #CarHistory #DidYouKnow', 'Discover the fascina-1713728011']
["Discover the story behind 'www' and how it changed the internet! #DidYouKnow #WorldWideWeb #TechHistory #Innovation", 'Discover the story b-1713728126']
['Chicken pox is more than just itchy spots. Learn the facts. #DidYouKnow #ChickenPox #VaccinesMatter', 'Chicken pox is more -1713728094']
["Meet Megalodon, the ocean's biggest shark! 🦈 Find out how it ruled the seas. #Did

chunk:   0%|          | 0/1655 [00:00<?, ?it/s, now=None]



MoviePy - Writing audio in Discover the fascinating history behind Mercedes-Benz, from its name to its innovative safety features! #MercedesBenz #CarHistory #DidYouKnowTEMP_MPY_wvf_snd.mp3


chunk:   0%|          | 0/1435 [00:00<?, ?it/s, now=None]





















chunk:   0%|          | 2/1655 [00:00<04:20,  6.35it/s, now=None]












chunk:   2%|▏         | 28/1655 [00:00<00:19, 83.92it/s, now=None]




















chunk:   4%|▎         | 60/1655 [00:00<00:10, 155.30it/s, now=None]




































chunk:   5%|▌         | 83/1655 [00:00<00:10, 144.29it/s, now=None]

























chunk:   6%|▌         | 102/1655 [00:00<00:10, 141.80it/s, now=None]







chunk:   8%|▊         | 125/1655 [00:00<00:09, 163.59it/s, now=None]


















chunk:   9%|▉         | 154/1655 [00:01<00:07, 196.97it/s, now=None]

















chunk:  11%|█         | 177/1655 [00:01<00:08, 166.37it/s, now=None]









chunk:  12%|█▏        | 205/1655 [00:01<00:07, 193.65it/s, now=None]















chunk:  14%|█▎        | 227/1655 [00:01<00:07, 190.18it/s, now=None]




















chunk:  15%|█▍        | 248/1655 [00:01<00:08, 175.42it/s, 

MoviePy - Done.
Moviepy - Writing video d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Discover the hidden facts about your favorite fictional characters! #DidYouKnow #FictionalCharacters #FunFacts.mp4



...























































































































































































































































































































































































































































































                                                 


                                                       






                                           


















                                                 


                                                       






                                           






t:  86%|████████▋ | 1970/2281 [15:00<01:04,  4.83it/s, now=None]

Moviepy - Done !
Moviepy - video ready d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Venus Earth's twin with a deadly twist. Discover why it's the hottest planet! #DidYouKnow #Venus #SpaceFacts.mp4


Moviepy - Done !
Moviepy - video ready d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Meet Megalodon, the ocean's biggest shark! 🦈 Find out how it ruled the seas. #DidYouKnow #Megalodon #Prehistoric.mp4


Moviepy - Done !
Moviepy - video ready d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Chicken pox is more than just itchy spots. Learn the facts. #DidYouKnow #ChickenPox #VaccinesMatter.mp4


Moviepy - Done !
Moviepy - video ready d:\Python Scripts\Ongoing\VideoCreationTiktok\media\Spices aren't just for cooking—they've shaped history! 🌶️🧭 #DidYouKnow #SpiceTrade #History #FoodCulture.mp4


In [1]:
from __utils__.VideoProcessor import VideoProcessor


def processVideo(title):
    # Get most recent .fdmdownload file
    recent_fdmdownload = VideoProcessor.get_most_recent_fdmdownload()
    recent_mp4 = VideoProcessor.convert_fdmdownload_to_mp4(recent_fdmdownload)
    current_directory = os.path.join(os.getcwd(), "media")

    title.replace("?", "").replace(":", "")

    # Crop video
    path = os.path.join(current_directory, title + ".mp4")
    VideoProcessor.cropVideo(recent_mp4, path, 69, 244, 1676, 942)
    os.remove(recent_mp4)

In [4]:
processVideo("Do you know! Cholera can be deadly, but we can fight it with clean water and sanitation. 💧 #DidYouKnow #Cholera #PublicHealth")

NameError: name 'os' is not defined